In [5]:
from zumi.util.camera import Camera
from zumi.zumi import Zumi
import cv2
import PIL.Image
import IPython.display
import time
import math
import pyzbar.pyzbar as pyzbar
import numpy as np

#height and width of the camera in pixels
width = 160*2
height = 128*2

zumi = Zumi()

try:
    camera = Camera(width,height)
    camera.start_camera()
except:
    camera.close()

def find_qr_area(frame):
    # Run a function that looks for codes in that frame
    decodedObjects = pyzbar.decode(frame) 
    
    # If the code finds more than one code...
    if len(decodedObjects) > 0:
        
        # For each QR code...
        for obj in decodedObjects: 
            #width of qr code
            width = obj.rect[2]
            #height of qr code
            height = obj.rect[3]
            #area in pixels
            area_of_qr = width*height
            return area_of_qr
 
def calc_distance(area_detected,area_of_object):
    #cant see beyond 10 inches and below 4 inches
    
    #when you are below 4 inches the area is too big
    #for the QR code to be seen properly
    
    #when you are too far away you see nothing
    print(area_detected)

    if area_detected:
        k = 0.8 #some constant which we can vary to get better results
        distance = ( k*(area_of_object*width*height)/area_detected )**(1/2)
    else:
        distance = -1
    return distance  

def keep_distance(distance):
    if distance < 5:
        zumi.reverse(40,0.3)
    elif distance > 8:
        zumi.forward(40,0.3)
    else:
        zumi.stop()


length_of_qr = 3 #3 inches
area_of_qr = length_of_qr**2 #9 inches squared


try: 
    while True:
        time_start = time.time()
        image = camera.capture()
        area = find_qr_area(image)
        distance = calc_distance(area,area_of_qr)
        print(distance)
        keep_distance(distance)
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert it to gray
        ret,black_white = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)
        IPython.display.display(PIL.Image.fromarray(black_white))
        
        print("FPS")
        print(1/(time.time()-time_start))
        
        IPython.display.clear_output(wait=True) 

finally:
    camera.close()
    zumi.stop()
    print("done")

Closing PiCamera
done


KeyboardInterrupt: 